In [2]:
"""
YouTube Video Transcription GUI Application

Created by: Hamed Ghane
Date: December 07, 2024

This script creates a GUI application that:
1. Takes a YouTube URL from the user
2. Attempts to retrieve the video's transcript from YouTube directly
3. If no transcript is found, it downloads the video's audio and uses OpenAI's Whisper model to transcribe it
4. Allows the user to choose between Persian or English transcription
5. Displays the transcript in the GUI with progress and status updates

Requirements:
- PyQt5 for the GUI (pip install PyQt5)
- openai for Whisper transcription (pip install openai)
- youtube_transcript_api to fetch existing transcripts (pip install youtube-transcript-api)
- yt-dlp to download audio (pip install yt-dlp)
- googletrans for translation (pip install googletrans==3.1.0a0)
- A file named 'sk.py' containing `Hamedkey = "<YOUR_OPENAI_API_KEY>"` or replace the import with your key directly

The script also logs its actions to a log file in the user's Documents folder and prints logs to the console.

Usage:
- Run this script
- Enter the YouTube URL in the provided text box
- Select the desired language (English or Persian)
- Click "Transcribe" to retrieve or generate the transcript
- Use the "Exit" button to safely close the application
"""

import sys
import os
import openai
import logging
from pathlib import Path
from PyQt5.QtWidgets import (QApplication, QMainWindow, QPushButton, QVBoxLayout, 
                           QWidget, QLabel, QTextEdit, QProgressBar, QLineEdit, 
                           QHBoxLayout, QComboBox, QMessageBox)
from PyQt5.QtCore import Qt, QTimer
from youtube_transcript_api import YouTubeTranscriptApi
import subprocess
import re
from typing import Optional, Tuple
from googletrans import Translator
import time

# Set up logging
log_path = os.path.join(os.path.expanduser('~'), 'Documents', 'transcription_app.log')
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_path),
        logging.StreamHandler()
    ]
)

def get_video_id(url: str) -> Optional[str]:
    """Extract YouTube video ID from various URL formats"""
    patterns = [
        r"(?:v=|\/)([0-9A-Za-z_-]{11}).*",
        r"(?:youtu\.be\/)([0-9A-Za-z_-]{11})",
        r"(?:embed\/)([0-9A-Za-z_-]{11})",
        r"(?:shorts\/)([0-9A-Za-z_-]{11})"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def download_audio(video_id: str, progress_callback=None) -> Tuple[bool, str]:
    """Download audio from YouTube video"""
    try:
        output_path = os.path.join(os.path.expanduser('~'), 'Documents', f'audio_{video_id}.mp3')
        command = [
            'yt-dlp',
            '--extract-audio',
            '--audio-format', 'mp3',
            '-o', output_path,
            '--newline',
            f'https://www.youtube.com/watch?v={video_id}'
        ]
        
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            creationflags=subprocess.CREATE_NO_WINDOW
        )
        
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output and progress_callback:
                if '[download]' in output and '%' in output:
                    try:
                        percent = float(output.split('%')[0].split()[-1])
                        progress_callback(percent)
                    except (ValueError, IndexError):
                        pass
        
        if process.returncode == 0:
            return True, output_path
        return False, process.stderr.read()
        
    except Exception as e:
        return False, str(e)

def translate_text(text: str, target_lang: str = 'fa') -> str:
    """Translate text using Google Translate"""
    try:
        translator = Translator()
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        translated_chunks = []
        
        for chunk in chunks:
            for attempt in range(3):
                try:
                    translation = translator.translate(chunk, dest=target_lang)
                    translated_chunks.append(translation.text)
                    break
                except Exception as e:
                    if attempt == 2:
                        raise
                    time.sleep(1)
        
        return ' '.join(translated_chunks)
    except Exception as e:
        logging.error(f"Translation error: {e}")
        return f"Translation failed: {str(e)}"

def whisper_transcribe(audio_file_path: str, language: str) -> str:
    """Transcribe audio using OpenAI Whisper"""
    try:
        from sk import Hamedkey
        openai.api_key = Hamedkey
    except ImportError:
        raise ImportError("OpenAI API key not found. Please create sk.py with your API key.")
    
    lang_code = 'fa' if language == 'Persian' else 'en'
    
    for attempt in range(3):
        try:
            with open(audio_file_path, "rb") as audio:
                transcript = openai.Audio.transcribe(
                    model="whisper-1",
                    file=audio,
                    language=lang_code,
                    response_format="text"
                )
            return transcript
        except Exception as e:
            if attempt == 2:
                raise
            time.sleep(1)

def create_window():
    """Create and configure the main application window"""
    window = QMainWindow()
    window.setWindowTitle('YouTube Audio Transcription')
    window.setGeometry(100, 100, 800, 600)
    
    central_widget = QWidget()
    window.setCentralWidget(central_widget)
    main_layout = QVBoxLayout(central_widget)
    
    # Header
    welcome_label = QLabel('YouTube Audio Transcription')
    welcome_label.setAlignment(Qt.AlignCenter)
    welcome_label.setStyleSheet('font-size: 16px; margin: 10px; font-weight: bold;')
    main_layout.addWidget(welcome_label)
    
    # Input section
    input_layout = QHBoxLayout()
    url_input = QLineEdit()
    url_input.setPlaceholderText("Enter YouTube URL here...")
    input_layout.addWidget(url_input)
    
    language_combo = QComboBox()
    language_combo.addItems(["English", "Persian"])
    language_combo.setToolTip("Select transcription language")
    input_layout.addWidget(language_combo)
    main_layout.addLayout(input_layout)
    
    # Status and progress
    status_label = QLabel('')
    status_label.setAlignment(Qt.AlignCenter)
    main_layout.addWidget(status_label)
    
    progress_bar = QProgressBar()
    progress_bar.setRange(0, 100)
    progress_bar.hide()
    main_layout.addWidget(progress_bar)
    
    # Output area
    transcript_text = QTextEdit()
    transcript_text.setPlaceholderText("Transcription will appear here...")
    transcript_text.setReadOnly(True)
    main_layout.addWidget(transcript_text)
    
    # Button layout
    button_layout = QHBoxLayout()
    
    # Transcribe button (larger)
    transcribe_btn = QPushButton('Transcribe')
    transcribe_btn.setStyleSheet('''
        QPushButton {
            padding: 12px;
            font-size: 14px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            min-width: 200px;
        }
        QPushButton:hover {
            background-color: #45a049;
        }
        QPushButton:disabled {
            background-color: #cccccc;
            color: #666666;
        }
    ''')
    
    # Exit button (smaller)
    exit_btn = QPushButton('Exit')
    exit_btn.setFixedWidth(100)
    exit_btn.setStyleSheet('''
        QPushButton {
            padding: 8px;
            font-size: 12px;
            background-color: #f44336;
            color: white;
            border: none;
            border-radius: 4px;
            min-width: 80px;
        }
        QPushButton:hover {
            background-color: #d32f2f;
        }
    ''')
    
    # Add buttons to layout
    button_layout.addWidget(transcribe_btn, alignment=Qt.AlignCenter)
    button_layout.addSpacing(40)
    button_layout.addWidget(exit_btn, alignment=Qt.AlignRight)
    button_layout.addSpacing(10)
    main_layout.addLayout(button_layout)
    
    def update_progress(percent):
        """Update progress bar value"""
        progress_bar.setValue(int(percent))
        QApplication.processEvents()
    
    def update_status(message: str, color: str = 'black'):
        """Update status label with message and color"""
        status_label.setText(message)
        status_label.setStyleSheet(f'color: {color};')
        QApplication.processEvents()
    
    def process_transcription(video_id: str, target_language: str):
        """Process the transcription request"""
        audio_file = None
        try:
            update_status("Checking for YouTube transcript...", 'blue')
            
            try:
                transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
                transcript = transcript_list.find_transcript(['en'])
                text = " ".join([entry['text'] for entry in transcript.fetch()])
                
                if target_language == "Persian":
                    update_status("Translating to Persian...", 'blue')
                    text = translate_text(text, 'fa')
                
                transcript_text.setText(text)
                update_status("Transcript retrieved and translated successfully!", 'green')
                
            except Exception as yt_error:
                update_status("No transcript available. Downloading audio...", 'blue')
                progress_bar.show()
                
                success, result = download_audio(video_id, update_progress)
                
                if success:
                    audio_file = result
                    update_status("Transcribing with Whisper...", 'blue')
                    text = whisper_transcribe(audio_file, target_language)
                    transcript_text.setText(text)
                    update_status("Transcription completed!", 'green')
                else:
                    raise Exception(f"Failed to download audio: {result}")
                    
        except Exception as e:
            update_status(f"Error: {str(e)}", 'red')
            logging.error(f"Transcription error: {e}")
        
        finally:
            if audio_file and os.path.exists(audio_file):
                try:
                    os.remove(audio_file)
                except Exception as e:
                    logging.warning(f"Could not delete audio file: {e}")
            
            progress_bar.hide()
            transcribe_btn.setEnabled(True)
    
    def start_transcription():
        """Initialize transcription process"""
        url = url_input.text().strip()
        if not url:
            update_status("Please enter a YouTube URL", 'red')
            return

        video_id = get_video_id(url)
        if not video_id:
            update_status("Invalid YouTube URL", 'red')
            return

        transcribe_btn.setEnabled(False)
        progress_bar.setValue(0)
        progress_bar.show()
        
        language = language_combo.currentText()
        QTimer.singleShot(100, lambda: process_transcription(video_id, language))
    
    def exit_application():
        """Safely exit the application"""
        try:
            # Clean up any remaining temporary files
            docs_path = os.path.join(os.path.expanduser('~'), 'Documents')
            for file in os.listdir(docs_path):
                if file.startswith('audio_') and file.endswith('.mp3'):
                    try:
                        file_path = os.path.join(docs_path, file)
                        if os.path.exists(file_path):
                            os.remove(file_path)
                    except Exception as e:
                        logging.warning(f"Could not delete file {file}: {e}")
        except Exception as e:
            logging.warning(f"Error during cleanup: {e}")
        
        try:
            # Close all windows and quit application
            QApplication.closeAllWindows()
            QApplication.quit()
        except Exception as e:
            logging.error(f"Error during application exit: {e}")
            sys.exit(0)
    
    # Connect button signals
    transcribe_btn.clicked.connect(start_transcription)
    exit_btn.clicked.connect(exit_application)
    window.closeEvent = lambda event: exit_application()
    
    return window

def main():
    """Main application entry point"""
    try:
        # Check if QApplication already exists
        app = QApplication.instance()
        if app is None:
            app = QApplication(sys.argv)
        
        # Create and show main window
        window = create_window()
        window.show()
        
        # Start application event loop
        return app.exec_()
    except Exception as e:
        QMessageBox.critical(None, "Error", f"An unexpected error occurred: {str(e)}")
        logging.error(f"Application error: {e}")
        return 1

if __name__ == '__main__':
    sys.exit(main())

SystemExit: 0

In [7]:
"""
YouTube Video Transcription GUI Application

Created by: Hamed Ghane
Date: December 07, 2024

This script creates a GUI application that:
1. Takes a YouTube URL from the user
2. Attempts to retrieve the video's transcript from YouTube directly
3. If no transcript is found, it downloads the video's audio and uses OpenAI's Whisper model to transcribe it
4. Allows the user to choose between Persian or English transcription
5. Displays the transcript in the GUI with progress and status updates

Requirements:
- PyQt5 for the GUI (pip install PyQt5)
- openai for Whisper transcription (pip install openai)
- youtube_transcript_api to fetch existing transcripts (pip install youtube-transcript-api)
- yt-dlp to download audio (pip install yt-dlp)
- googletrans for translation (pip install googletrans==3.1.0a0)
- A file named 'sk.py' containing your OpenAI API key (see below)

Before running this script, create a file named 'sk.py' with this content:
    Hamedkey = "your-openai-api-key-here"  # Replace with your actual OpenAI API key

The script logs its actions to a log file in the user's Documents folder and prints logs to the console.
"""

# System and OS operations
import sys              # Required for system-level operations and argument handling
import os               # Required for file and directory operations
import openai           # OpenAI API interface for Whisper transcription
import logging          # Logging functionality for debugging and tracking
from pathlib import Path # Modern path manipulation

# PyQt5 imports for GUI components
from PyQt5.QtWidgets import (
    QApplication,      # Main application class
    QMainWindow,       # Main window widget
    QPushButton,       # Button widget
    QVBoxLayout,       # Vertical layout manager
    QWidget,           # Base widget class
    QLabel,           # Text label widget
    QTextEdit,        # Multi-line text editor
    QProgressBar,     # Progress indicator
    QLineEdit,        # Single line text input
    QHBoxLayout,      # Horizontal layout manager
    QComboBox,        # Dropdown selection box
    QMessageBox       # Message dialog
)
from PyQt5.QtCore import Qt, QTimer  # Core Qt functionality and timer

# Additional functionality imports
from youtube_transcript_api import YouTubeTranscriptApi  # YouTube transcript access
import subprocess    # For running external commands (yt-dlp)
import re            # Regular expressions for URL parsing
from typing import Optional, Tuple  # Type hints for better code documentation
from googletrans import Translator  # Google Translate functionality
import time         # Time-related functions

# Configure logging to both file and console
log_path = os.path.join(os.path.expanduser('~'), 'Documents', 'transcription_app.log')
logging.basicConfig(
    level=logging.INFO,    # Set logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log format with timestamp
    handlers=[
        logging.FileHandler(log_path),     # Log to file
        logging.StreamHandler()            # Log to console
    ]
)

def get_video_id(url: str) -> Optional[str]:
    """
    Extract YouTube video ID from various URL formats.
    
    Args:
        url (str): YouTube URL in any standard format
        
    Returns:
        Optional[str]: The video ID if found, None otherwise
        
    Supports formats:
        - Standard: youtube.com/watch?v=VIDEO_ID
        - Short: youtu.be/VIDEO_ID
        - Embedded: youtube.com/embed/VIDEO_ID
        - Shorts: youtube.com/shorts/VIDEO_ID
    """
    patterns = [
        r"(?:v=|\/)([0-9A-Za-z_-]{11}).*",      # Standard and watch URLs
        r"(?:youtu\.be\/)([0-9A-Za-z_-]{11})",  # Short URLs
        r"(?:embed\/)([0-9A-Za-z_-]{11})",      # Embedded URLs
        r"(?:shorts\/)([0-9A-Za-z_-]{11})"      # Shorts URLs
    ]
    
    # Try each pattern until a match is found
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)  # Return the first capture group (video ID)
    return None  # Return None if no pattern matches

def download_audio(video_id: str, progress_callback=None) -> Tuple[bool, str]:
    """
    Download audio from YouTube video using yt-dlp.
    
    Args:
        video_id (str): YouTube video identifier
        progress_callback (function): Optional callback for progress updates
        
    Returns:
        Tuple[bool, str]: (Success status, Output path or error message)
        
    Downloads audio in MP3 format to user's Documents folder.
    """
    try:
        # Construct output path in user's Documents folder
        output_path = os.path.join(os.path.expanduser('~'), 'Documents', f'audio_{video_id}.mp3')
        
        # Prepare yt-dlp command with options
        command = [
            'yt-dlp',
            '--extract-audio',              # Extract only audio
            '--audio-format', 'mp3',        # Convert to MP3 format
            '-o', output_path,              # Set output path
            '--newline',                    # Enable progress tracking
            f'https://www.youtube.com/watch?v={video_id}'  # Video URL
        ]
        
        # Start download process without showing command window
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            creationflags=subprocess.CREATE_NO_WINDOW
        )
        
        # Monitor download progress
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output and progress_callback:
                if '[download]' in output and '%' in output:
                    try:
                        # Extract progress percentage and update UI
                        percent = float(output.split('%')[0].split()[-1])
                        progress_callback(percent)
                    except (ValueError, IndexError):
                        pass
        
        # Check if download was successful
        if process.returncode == 0:
            return True, output_path
        return False, process.stderr.read()
        
    except Exception as e:
        return False, str(e)

def translate_text(text: str, target_lang: str = 'fa') -> str:
    """
    Translate text using Google Translate with chunking and retry logic.
    
    Args:
        text (str): Text to translate
        target_lang (str): Target language code (default: 'fa' for Persian)
        
    Returns:
        str: Translated text or error message
        
    Handles long text by splitting into chunks and implements retry logic.
    """
    try:
        translator = Translator()
        # Split text into 1000-character chunks to avoid length limits
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        translated_chunks = []
        
        # Process each chunk with retry mechanism
        for chunk in chunks:
            for attempt in range(3):  # Try up to 3 times
                try:
                    translation = translator.translate(chunk, dest=target_lang)
                    translated_chunks.append(translation.text)
                    break  # Break retry loop if successful
                except Exception as e:
                    if attempt == 2:  # Last attempt failed
                        raise
                    time.sleep(1)  # Wait before retry
        
        # Join all translated chunks
        return ' '.join(translated_chunks)
    except Exception as e:
        logging.error(f"Translation error: {e}")
        return f"Translation failed: {str(e)}"

def whisper_transcribe(audio_file_path: str, language: str) -> str:
    """
    Transcribe audio using OpenAI's Whisper API.
    
    Args:
        audio_file_path (str): Path to the audio file
        language (str): Target language ('Persian' or 'English')
        
    Returns:
        str: Transcribed text
        
    Requires:
        - sk.py file with OpenAI API key (Hamedkey variable)
        - Valid audio file at specified path
    """
    try:
        # Import API key from sk.py - REPLACE WITH YOUR OWN API KEY IN sk.py
        from sk import Hamedkey  # Create sk.py with: Hamedkey = "your-openai-api-key-here"
        openai.api_key = Hamedkey
    except ImportError:
        raise ImportError("OpenAI API key not found. Please create sk.py with your API key.")
    
    # Set appropriate language code
    lang_code = 'fa' if language == 'Persian' else 'en'
    
    # Attempt transcription with retry mechanism
    for attempt in range(3):
        try:
            with open(audio_file_path, "rb") as audio:
                # Send request to OpenAI's Whisper API
                transcript = openai.Audio.transcribe(
                    model="whisper-1",
                    file=audio,
                    language=lang_code,
                    response_format="text"
                )
            return transcript
        except Exception as e:
            if attempt == 2:  # Last attempt failed
                raise
            time.sleep(1)  # Wait before retry

def create_window():
    """
    Create and configure the main application window with all UI elements.
    
    Returns:
        QMainWindow: Configured main window with all widgets and event handlers
        
    Creates:
        - URL input field
        - Language selection dropdown
        - Status display
        - Progress bar
        - Transcript display area
        - Transcribe and Exit buttons
    """
    # Create main window and set basic properties
    window = QMainWindow()
    window.setWindowTitle('YouTube Audio Transcription')
    window.setGeometry(100, 100, 800, 600)  # Position and size: x, y, width, height
    
    # Create central widget and main layout container
    central_widget = QWidget()
    window.setCentralWidget(central_widget)
    main_layout = QVBoxLayout(central_widget)  # Vertical layout for stacking elements
    
    # Create and configure header label
    welcome_label = QLabel('YouTube Audio Transcription')
    welcome_label.setAlignment(Qt.AlignCenter)  # Center align text
    welcome_label.setStyleSheet('font-size: 16px; margin: 10px; font-weight: bold;')  # Style header
    main_layout.addWidget(welcome_label)
    
    # Create input section with URL field and language selector
    input_layout = QHBoxLayout()  # Horizontal layout for input elements
    
    # URL input field
    url_input = QLineEdit()
    url_input.setPlaceholderText("Enter YouTube URL here...")  # Helper text
    input_layout.addWidget(url_input)
    
    # Language selection dropdown
    language_combo = QComboBox()
    language_combo.addItems(["English", "Persian"])  # Available languages
    language_combo.setToolTip("Select transcription language")  # Hover text
    input_layout.addWidget(language_combo)
    main_layout.addLayout(input_layout)
    
    # Status display and progress tracking
    status_label = QLabel('')  # Empty initially, will show status messages
    status_label.setAlignment(Qt.AlignCenter)
    main_layout.addWidget(status_label)
    
    # Progress bar for download/transcription progress
    progress_bar = QProgressBar()
    progress_bar.setRange(0, 100)  # Percentage scale
    progress_bar.hide()  # Hidden initially
    main_layout.addWidget(progress_bar)
    
    # Transcript display area
    transcript_text = QTextEdit()
    transcript_text.setPlaceholderText("Transcription will appear here...")
    transcript_text.setReadOnly(True)  # Prevent user editing
    main_layout.addWidget(transcript_text)
    
    # Create button layout for action buttons
    button_layout = QHBoxLayout()
    
    # Configure Transcribe button (larger, green)
    transcribe_btn = QPushButton('Transcribe')
    transcribe_btn.setStyleSheet('''
        QPushButton {
            padding: 12px;
            font-size: 14px;
            background-color: #4CAF50;  /* Green */
            color: white;
            border: none;
            border-radius: 5px;
            min-width: 200px;
        }
        QPushButton:hover {
            background-color: #45a049;  /* Darker green on hover */
        }
        QPushButton:disabled {
            background-color: #cccccc;  /* Gray when disabled */
            color: #666666;
        }
    ''')
    
    # Configure Exit button (smaller, red)
    exit_btn = QPushButton('Exit')
    exit_btn.setFixedWidth(100)  # Make button narrower
    exit_btn.setStyleSheet('''
        QPushButton {
            padding: 8px;
            font-size: 12px;
            background-color: #f44336;  /* Red */
            color: white;
            border: none;
            border-radius: 4px;
            min-width: 80px;
        }
        QPushButton:hover {
            background-color: #d32f2f;  /* Darker red on hover */
        }
    ''')
    
    # Add buttons to layout with proper spacing and alignment
    button_layout.addWidget(transcribe_btn, alignment=Qt.AlignCenter)
    button_layout.addSpacing(40)  # Space between buttons
    button_layout.addWidget(exit_btn, alignment=Qt.AlignRight)
    button_layout.addSpacing(10)  # Right margin
    main_layout.addLayout(button_layout)
    
    # Helper function for updating progress bar
    def update_progress(percent):
        """Update progress bar and keep UI responsive"""
        progress_bar.setValue(int(percent))
        QApplication.processEvents()  # Prevent UI freezing
    
    # Helper function for updating status messages
    def update_status(message: str, color: str = 'black'):
        """Update status label with message and color"""
        status_label.setText(message)
        status_label.setStyleSheet(f'color: {color};')
        QApplication.processEvents()  # Prevent UI freezing

    def process_transcription(video_id: str, target_language: str):
        """
        Handle the complete transcription process workflow.
        
        Args:
            video_id (str): YouTube video identifier
            target_language (str): Desired transcription language
            
        Process:
            1. Try to fetch existing YouTube transcript
            2. If not available, download audio and use Whisper
            3. Translate if needed
            4. Update UI with progress and results
        """
        audio_file = None
        try:
            update_status("Checking for YouTube transcript...", 'blue')
            
            try:
                # First attempt: Get existing transcript from YouTube
                transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
                transcript = transcript_list.find_transcript(['en'])  # Try English first
                text = " ".join([entry['text'] for entry in transcript.fetch()])
                
                # Translate to Persian if requested
                if target_language == "Persian":
                    update_status("Translating to Persian...", 'blue')
                    text = translate_text(text, 'fa')
                
                # Display results
                transcript_text.setText(text)
                update_status("Transcript retrieved and translated successfully!", 'green')
                
            except Exception as yt_error:
                # Second attempt: Download and transcribe with Whisper
                update_status("No transcript available. Downloading audio...", 'blue')
                progress_bar.show()
                
                # Download the audio
                success, result = download_audio(video_id, update_progress)
                
                if success:
                    audio_file = result
                    update_status("Transcribing with Whisper...", 'blue')
                    text = whisper_transcribe(audio_file, target_language)
                    transcript_text.setText(text)
                    update_status("Transcription completed!", 'green')
                else:
                    raise Exception(f"Failed to download audio: {result}")
                    
        except Exception as e:
            # Handle any errors that occurred
            update_status(f"Error: {str(e)}", 'red')
            logging.error(f"Transcription error: {e}")
        
        finally:
            # Clean up temporary files and reset UI
            if audio_file and os.path.exists(audio_file):
                try:
                    os.remove(audio_file)
                except Exception as e:
                    logging.warning(f"Could not delete audio file: {e}")
            
            progress_bar.hide()
            transcribe_btn.setEnabled(True)
    
    def start_transcription():
        """
        Initialize the transcription process after validation.
        
        Validates:
            - URL is not empty
            - URL contains valid video ID
        Then starts the transcription process in a non-blocking way.
        """
        # Validate URL input
        url = url_input.text().strip()
        if not url:
            update_status("Please enter a YouTube URL", 'red')
            return

        # Extract and validate video ID
        video_id = get_video_id(url)
        if not video_id:
            update_status("Invalid YouTube URL", 'red')
            return

        # Prepare UI for transcription
        transcribe_btn.setEnabled(False)
        progress_bar.setValue(0)
        progress_bar.show()
        
        # Start transcription process non-blocking
        language = language_combo.currentText()
        QTimer.singleShot(100, lambda: process_transcription(video_id, language))
    
    def exit_application():
        """
        Safely exit the application with cleanup.
        
        Performs:
            1. Cleanup of temporary files
            2. Proper closing of all windows
            3. Clean termination of QApplication
        """
        try:
            # Clean up any remaining temporary files
            docs_path = os.path.join(os.path.expanduser('~'), 'Documents')
            for file in os.listdir(docs_path):
                if file.startswith('audio_') and file.endswith('.mp3'):
                    try:
                        file_path = os.path.join(docs_path, file)
                        if os.path.exists(file_path):
                            os.remove(file_path)
                    except Exception as e:
                        logging.warning(f"Could not delete file {file}: {e}")
        except Exception as e:
            logging.warning(f"Error during cleanup: {e}")
        
        try:
            # Properly close application
            QApplication.closeAllWindows()
            QApplication.quit()
        except Exception as e:
            logging.error(f"Error during application exit: {e}")
            sys.exit(0)
    
    # Connect button signals to their handlers
    transcribe_btn.clicked.connect(start_transcription)
    exit_btn.clicked.connect(exit_application)
    window.closeEvent = lambda event: exit_application()
    
    return window

def main():
    """
    Main application entry point.
    
    Handles:
        - QApplication initialization
        - Window creation and display
        - Error handling and logging
        - Application execution
    """
    try:
        # Initialize or get existing QApplication instance
        app = QApplication.instance()
        if app is None:
            app = QApplication(sys.argv)
        
        # Create and display main window
        window = create_window()
        window.show()
        
        # Start event loop
        return app.exec_()
    except Exception as e:
        # Handle any unhandled exceptions
        QMessageBox.critical(None, "Error", f"An unexpected error occurred: {str(e)}")
        logging.error(f"Application error: {e}")
        return 1

# Application entry point
if __name__ == '__main__':
    sys.exit(main())

SystemExit: 0